In [1]:
import numpy as np
import pandas as pd

#import warnings
#warnings.filterwarnings('ignore')

In [2]:
money_to_spend = 5000
min_buy_val = 20
max_buy_val = 100
days_in_future = 2
capital_gains_tax = .24
stock_trading_days = 250

df = pd.read_csv('stocks to buy.csv')

if 'Unnamed: 1' in df.columns:
    df = df.drop(columns=['Unnamed: 1'])
    
if 'Symbol.1' in df.columns:
    df = df.drop(columns=['Symbol.1'])

if 'High slope 20 days' in df.columns:
    df_type = 'regression'


In [3]:
#group the df into symbols
#take the first item in the df for each symbol and put it into the buy price
#take the rest and put it into the future prices
df_copy = df.copy()

buy_price = df_copy.groupby('Symbol').apply(lambda group: group.iloc[0:1,:])
buy_price = buy_price[(buy_price['Open slope'] > 0) & (buy_price['Volume'] > 0)]
avg_lin_reg_mse = buy_price['Open Linear reg MSE'].describe()[1]

buy_price = buy_price[buy_price['Open Linear reg MSE'] <= avg_lin_reg_mse]
buy_price = buy_price[(buy_price['Open'] >= min_buy_val) & (buy_price['Open'] <= max_buy_val)]

future_prices = df_copy.groupby('Symbol').apply(lambda group: group.iloc[1:,:])

In [4]:
#buy a bunch of random stocks that meet the above criteria
#set error bounds to be the high and low price for the day

money_spent = money_to_spend
stocks_to_buy = {}
buy_price['number bought'] = 0

while money_spent > 0:
    the_sample = buy_price.sample().index[0]
    buy_price.at[the_sample, 'number bought'] += 1
    money_spent -= buy_price.at[the_sample, 'Open']   

buy_price['money spent high'] = buy_price['number bought'] * buy_price['High']
buy_price['money spent low'] = buy_price['number bought'] * buy_price['Low']

In [5]:
#cleanup the df and add the projection for the price
buy_data = buy_price.drop(['High', 'Low', 'Close', 'Volume', 'Dividends'], axis=1)
buy_data = buy_data[buy_data['number bought'] > 0]
buy_data['projection'] = buy_data['Open slope'] * days_in_future + buy_data['Open']
    
future_prices = future_prices[future_prices['Symbol'].isin(buy_data['Symbol'].unique())]
future_prices = future_prices.drop(['Volume', 'Dividends'], axis=1)
#display(buy_data)

In [6]:
total_bad_projections = 0
average_days = 0
money_made_high = 0
money_made_low = 0

for symbol in buy_data['Symbol'].unique():
    
    try:
        
        the_projection = buy_data[buy_data['Symbol'] == symbol]['projection'][0]
        temp_df = future_prices[future_prices['Symbol'] == symbol]['High'].to_frame()
        
        first_index = buy_data[buy_data['Symbol'] == symbol].index[0][1]
        index_beats_proj = temp_df[temp_df['High'] >= buy_data[buy_data['Symbol'] == symbol]['projection'][0]].index[0][1]
        days_between = index_beats_proj - first_index
        
        curr_mon_spent_high = buy_data[buy_data['Symbol'] == symbol]['money spent high'][0]
        curr_mon_spent_low = buy_data[buy_data['Symbol'] == symbol]['money spent low'][0]
        num_bought = buy_data[buy_data['Symbol'] == symbol]['number bought'][0]
        
        sell_price = temp_df[temp_df['High'] >= buy_data[buy_data['Symbol'] == symbol]['projection'][0]].iloc[0, 0]
        
        if days_between <= days_in_future:
            money_made_high += num_bought * sell_price - curr_mon_spent_high
            money_made_low += num_bought * sell_price - curr_mon_spent_low
        
        elif days_between > days_in_future:
            high_low_avg = (future_prices[future_prices['Symbol'] == symbol]['High'] + future_prices[future_prices['Symbol'] == symbol]['Low']) / 2 
            money_made_high += high_low_avg[days_in_future-1] * num_bought - curr_mon_spent_high
            money_made_low += high_low_avg[days_in_future-1] * num_bought - curr_mon_spent_low
            days_between = days_in_future
        
        average_days += days_between
      
    
    except:
        high_low_avg = (future_prices[future_prices['Symbol'] == symbol]['High'] - future_prices[future_prices['Symbol'] == symbol]['Low']) / 2 + future_prices[future_prices['Symbol'] == symbol]['Low']
        money_made_high += high_low_avg[days_in_future-1]*num_bought - curr_mon_spent_high 
        money_made_low += high_low_avg[days_in_future-1]*num_bought - curr_mon_spent_low
        average_days += days_in_future
        
        total_bad_projections +=1

average_days_bet = average_days/len(buy_data) 
avg_money_made = (money_made_high+money_made_low)/2
if avg_money_made > 0:
    avg_money_made = avg_money_made - (avg_money_made * capital_gains_tax)
    
money_spent_high = buy_price['money spent high'].sum()
money_spent_low = buy_price['money spent low'].sum()
avg_money_spent = (money_spent_high + money_spent_low)/2    
daily_perc_gain = avg_money_made/avg_money_spent/average_days_bet
future_value_low = money_spent_low * ((1 + daily_perc_gain) ** (stock_trading_days))
future_value_high = money_spent_high * ((1 + daily_perc_gain) ** (stock_trading_days))

print("average days between buy and sell", average_days_bet)
print("total bad projections", total_bad_projections)
print("Money actually spent low to high and average = ", money_spent_low, money_spent_high, avg_money_spent)
print("Total money made low to high and average = ", money_made_low, money_made_high, avg_money_made)
print("Average daily percent gain = ", daily_perc_gain)
print("Future value high", future_value_high)
print("Future value low", future_value_low)
print("Number of stocks bought:", buy_data['number bought'].sum())

display(buy_data)

average days between buy and sell 1.1147540983606556
total bad projections 1
Money actually spent low to high and average =  4982.199143962191 5088.84973611243 5035.52444003731
Total money made low to high and average =  198.99527506167394 92.58385018320556 110.80006759305421
Average daily percent gain =  0.01973859514844656
Future value high 674269.9946134514
Future value low 660138.8455476118
Number of stocks bought: 132


,,Symbol,Open,Stock Splits,Open slope,Open Linear reg MSE,number bought,money spent high,money spent low,projection
Symbol,,,,,,,,,,
ACIW,330,ACIW,34.759998,0.0,0.040872,0.669539,1,35.099998,34.380001,34.841743
ADC,570,ADC,58.874018,0.0,0.095423,0.649518,1,59.786484,58.874018,59.064864
ADM,600,ADM,59.840000,0.0,0.107303,1.251319,1,60.520000,59.349998,60.054606
AEM,720,AEM,67.166003,0.0,0.255210,1.400070,1,68.726847,66.987052,67.676422
AGNCO,1050,AGNCO,24.477878,0.0,0.015045,0.002682,2,49.002981,48.739315,24.507969
...,...,...,...,...,...,...,...,...,...,...
VRRM,40410,VRRM,26.639999,0.0,0.130301,0.641952,1,26.985001,26.180000,26.900601
VSH,40470,VSH,23.409099,0.0,0.073608,0.125272,1,23.478798,23.249785,23.556314
VTS,40650,VTS,24.406710,0.0,0.148896,0.193082,1,24.803870,24.406710,24.704503
